In [2]:
import torch
import numpy as np
import torch.nn as nn

In [3]:
# model = Network()
# for epoch in range(10):
#     for idx,input_data,target in enumerate(train_loader):
#         # input_data:
#         #     a batch containing input data with shape: [batch,window_size,state_size]
#         #     e.g. with half_window = 20, input_data is: [128,41,30]
#         # expand input_data as [128,1,41,30]
#         # as its channel is set as 1, in order to fit first CNN layer
#         # this expansion will be done in forward()
#         # target:
#         #     shape [128,30]
#         # output of input_data after model-forward should has same shape with target
#         out = model(input_data)
#         loss = criterion(out, target)
class Network(nn.Module):
    
    def __init__(self):
        super(Network,self).__init__()
        
        ###### parameters ######
        self.kernel = 6
        self.m = 30 # original data: [32768,30]
        self.drop_ratio = 0.2
        self.CNNhid = 100
        self.RNNhid = 100
        
        ###### CNN ######
        # in: [128,1,41,30] [batch,channel,window_size,state_size]
        # out: [128,100,36] [batch,cnn_hidden,seq_len]
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=self.CNNhid,kernel_size=(self.kernel,self.m)),
            nn.ReLU(),
            nn.Dropout(self.drop_ratio)
        )
        
        ###### GRU ######
        # in: [36,128,100]
        # out: [1,128,100] [layer_num,batch,gru_hidden]
        self.gru = nn.GRU(input_size=self.CNNhid, hidden_size=self.RNNhid)
        
        ###### output ######
        self.output = nn.Sequential(
            nn.Linear(self.RNNhid,self.m),
            nn.Sigmoid()
        )
        
    def forward(self,x):
        
        print("shape of input: {}".format(x.shape))
        ###### CNN ######
        # to fit CNN layer:
        # from: [128,41,30] [batch,window_size,state_size]
        # to: [128,1,41,30] [batch,channel,window_size,state_size]
        x = x.unsqueeze(1)
        print("shape before CNN: {}".format(x.shape))
        # in: [128,1,41,30] [batch,channel,window_size,state_size]
        # out: [128,100,36] [batch,cnn_hidden,seq_len]
        x = self.cnn(x)
        x = x.squeeze(3)
        print("shape after CNN: {}".format(x.shape))
        
        ###### GRU ######
        # to fit GRU:
        # from: [128,100,36] [batch,cnn_hidden,seq_len]
        # to: [36,128,100] [seq_len,batch,cnn_hidden]
        x = x.permute(2,0,1).contiguous()
        print("shape before GRU: {}".format(x.shape))
        # in: [36,128,100] [seq_len,batch,cnn_hidden]
        # out: [1,128,100] [layer_num,batch,gru_hidden]
        _, hidden = self.gru(x)
        self.gru_drop = nn.Dropout(self.drop_ratio)
        print("shape of GRU-hidden: {}".format(hidden.shape))
        hidden = self.gru_drop(hidden.squeeze(0))
        
        ###### output ######
        # in: [128,100] [batch,gru_hidden]
        # out: [128,30] [batch,state_size]
        print("shape of GRU-hidden before Linear and Sigmoid: {}".format(hidden.shape))
        output = self.output(hidden)
        
        return output

In [46]:
if __name__ == "__main__":
    x = torch.randn(128,41,30)
    model = Network()
    out = model(x)
    print("shape of output: {}".format(out.shape))

shape of input: torch.Size([128, 41, 30])
shape before CNN: torch.Size([128, 1, 41, 30])
shape after CNN: torch.Size([128, 100, 36])
shape before GRU: torch.Size([36, 128, 100])
shape of GRU-hidden: torch.Size([1, 128, 100])
shape of GRU-hidden before Linear and Sigmoid: torch.Size([128, 100])
shape of output: torch.Size([128, 30])
